<a href="https://colab.research.google.com/github/lizardnote/NLP/blob/main/%EA%B9%80%ED%9A%A8%EC%9E%AC_2%EC%B0%A8_%EB%AA%A8%EB%8D%B8(TRL_SFT%2C_datacollator).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting

In [ ]:
# 1. 환경 설정
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    DataCollatorForLanguageModeling)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from datasets import load_dataset
import torch


In [ ]:
from trl import SFTTrainer, DPOTrainer
from transformers import DataCollatorForLanguageModeling

In [ ]:
# 2. Hugging Face 로그인

from huggingface_hub import login
login("인증토큰")

UnicodeEncodeError: 'latin-1' codec can't encode characters in position 7-10: ordinal not in range(256)

 # Data load

In [ ]:
# 3. 데이터셋 불러오기
def load_jsonl_dataset(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = [json.loads(line) for line in f]
    return Dataset.from_list(data)

train_dataset = load_jsonl_dataset("train_dataset.jsonl")
eval_dataset = load_jsonl_dataset("test_dataset.jsonl")


# Load model and tokenizer
- BitsAndbytesConfig : 4/8bit 압축설정객체로 적은 vram으로 돌려볼 수 있음


In [ ]:
# 4. model 로드, tokenizer 설정
model_name = "meta-llama/Llama-2-7b-chat-hf"


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:935: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


### 1차 모델과 비교

```
# 토크나이징 함수
def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)

train_dataset = train_dataset.map(tokenize)
eval_dataset = eval_dataset.map(tokenize)

```


- 1차 모델에서 padding = 'max_length' 옵션을 설정했는데 주의할 점이 있음
-항상 512 토큰에 맞춰지기 때문에 학습 배치 자체 크게가 고정되면서 50토큰짜리고 512로 증가 (메모리 낭비)
- DataCollatorForCompletionOnlyLM이랑 충돌 가능성 있음(collator가 패딩 관리 역할인데 이미 적용한 상황)
- loss 계산을 위해서 label에 처리 필요함
- packing = True 도 사용 불가

-> DataCollatorForCompletionOnlyLM를 쓰면
  - 동적 패딩 + response 부분만 학습(loss mask 처리) + packing
  - 더 효율적이고 성능 좋은 모델을 얻을 수 있음.




In [ ]:
# 5. Collator 정의
from trl import DataCollatorForCompletionOnlyLM

collator = DataCollatorForCompletionOnlyLM(
    tokenizer=tokenizer,
    response_template="[/INST]",  # 응답은 [/INST] 뒤부터 시작
    instruction_template="[INST]"  # (선택 사항이지만 명시하면 더 안전)
)


IndentationError: unexpected indent (ipython-input-1-2844259787.py, line 2)

In [ ]:
# 6. training_args 설정
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    save_strategy="no",  # 체크포인트 저장 생략
    logging_steps=5,
    fp16=False,
    bf16=True,
    report_to="none"
)


eval_steps 지정 없음

eval_dataset이 있으니까 eval_steps=5 같이 지정해줘야 평가가 주기적으로 돌아감

save_total_limit & save_steps (만약 save_strategy 바꾸면)

지금은 저장 안 하니까 상관 없지만, 저장할 땐 이거 설정 필요

remove_unused_columns=False 고려

LoRA + PEFT 환경에서 인풋 외 컬럼 날리는 걸 막기 위해 유용함

In [ ]:
# 7. SFT Trainer 학습

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator,
    packing=True    # 패킹 사용
)


In [ ]:
trainer.train()

Step,Training Loss
5,2.237100
10,0.415800
15,0.380000
20,0.359200
25,0.336000
30,0.353300
35,0.254700
40,0.290800
45,0.223500
50,0.245200


TrainOutput(global_step=120, training_loss=0.2918899623056253, metrics={'train_runtime': 64.4054, 'train_samples_per_second': 3.726, 'train_steps_per_second': 1.863, 'total_flos': 4871462301204480.0, 'train_loss': 0.2918899623056253, 'epoch': 1.0})

In [ ]:
# 8. 모델 저장
trainer.save_model("./results")
tokenizer.save_pretrained("./results")

# 모델 폴더 압축
import shutil
shutil.make_archive("model_output", 'zip', "./results")  # 'model_output.zip' 생성

# 다운로드 (Colab이나 RunPod에서 수동 다운로드 가능)
from IPython.display import FileLink
FileLink("model_output.zip")

# # 압축 해제 후
# from transformers import AutoModelForCausalLM, AutoTokenizer
# model = AutoModelForCausalLM.from_pretrained("./results").to("cuda")
# tokenizer = AutoTokenizer.from_pretrained("./results")


model.save_pretrained("/workspace/first_model")
tokenizer.save_pretrained("/workspace/first_model")


In [ ]:
# 9. inference

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 모델과 토크나이저 불러오기 (학습된 모델일 경우 해당 경로로)
model_path = "./results"  # 또는 checkpoint 경로

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to("cuda")

# 텍스트 프롬프트 만들기 (SFT 학습한 포맷과 동일해야 함!)
instruction = "토익 공부 계획 짜줘"
prompt = f"<s>### Instruction\n{instruction}\n### Response"

# 토크나이징
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# 생성
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1,
        eos_token_id=tokenizer.eos_token_id
    )

# 출력 디코딩
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output_text)


### Inference
학습이 완료된 모델을 이용해서 새로운 입력에 대한 출력(응답)을 생성

```
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
instruction = "채용 공고를 요약해줘."
output = pipe(instruction, max_new_tokens=100, do_sample=True)[0]["generated_text"]
print(output)


prompt = "<s>[INST] 채용 공고를 요약해줘 [/INST]"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
output_ids = model.generate(input_ids, max_new_tokens=100)
print(tokenizer.decode(output_ids[0], skip_special_tokens=True))

```



In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 0.17247724533081055, 'eval_runtime': 3.1469, 'eval_samples_per_second': 19.066, 'eval_steps_per_second': 19.066, 'epoch': 1.0}


In [ ]:
def chat(prompt, max_new_tokens=256):
    formatted_prompt = tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt}],
        tokenize=False,
        add_generation_prompt=True
    )
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)
    output = model.generate(**inputs, max_new_tokens=max_new_tokens)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response


# from transformers import AutoTokenizer, AutoModelForCausalLM

# model_name = "meta-llama/Llama-2-7b-chat-hf"

# # tokenizer 로드
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.pad_token = tokenizer.eos_token  # 로드해야 사용 가능함

# # 모델 로드 (4bit 설정 포함 가능)
# model = AutoModelForCausalLM.from_pretrained(model_name)


### 참고

| 계열            | 대표 모델                          | 만든 곳             | 특징                            |
| ------------- | ------------------------------ | ---------------- | ----------------------------- |
| **LLaMA**     | LLaMA-2, LLaMA-3               | Meta             | 효율적·가벼움, HuggingFace에서 잘 지원   |
| **Mistral**   | Mistral-7B, Mixtral            | Mistral AI       | FlashAttention, 빠르고 가볍고 성능 좋음 |
| **Qwen**      | Qwen1.5-7B, Qwen2-7B, Qwen3-8B | Alibaba          | 중국어/영어 지원 강함, reasoning 우수    |
| **Gemma**     | Gemma-7B                       | Google           | 연구용 허용, Google Colab 최적화      |
| **Command-R** | Command-R+, R+                 | Cohere           | RAG 최적화 모델                    |
| **Yi**        | Yi-6B, Yi-34B                  | Tsinghua x 01.AI | Multilingual 강점 (한글도 꽤 잘함)    |
| **Phi**       | Phi-2                          | Microsoft        | 초경량 (1.3B), 초저사양에서 잘 돌아감      |
| **OpenChat**  | OpenChat-3.5                   | Community        | DPO + SFT 기반, 성능 대비 인기        |


| 목적         | 추천 계열                        |
| ---------- | ---------------------------- |
| 대화형 챗봇     | LLaMA / OpenChat / Yi        |
| 한국어 잘하는 모델 | Qwen / Yi / KoAlpaca (작은 모델) |
| 추론 잘하는 모델  | Qwen / Mixtral / GPT-4 계열    |
| RAG 특화     | Command-R / LLaMA-2 + RAG 튜닝 |
| 리소스 적을 때   | Phi / TinyLLaMA / Mistral-7B |
